In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys
from datetime import date, datetime, timedelta


##### Entering the search query, from and to date - to get the news articles in that period

In [2]:
def getQuery(str):
    result = str.split()
    return result
query = getQuery('citizenship amendment bill india')
# print(query)
start = date(2020, 1, 1)
end = date(2020, 1, 2)
print(start)


2020-01-01


In [3]:
# creating a function for getting a url
def createUrl(query, month, day, year):
    tempQ = []
    len_a = len(query)
    if (len_a > 1):
        for i in range(len_a):
            tempQ.append(query[i])        
        
        sep = '+'
        tempQ1 = sep.join(tempQ)
        url = 'https://www.google.com/search?q=' + str(tempQ1) + '&sxsrf=ACYBGNTwLQ0U4gEoAyzjmWThzJK06MJhxw%3A1577815826486&source=lnt&tbs=cdr%3A1%2Ccd_min%3A' + str(month) + '%2F' + str(day) + '%2F' + str(year) + '%2Ccd_max%3A' + str(month) + '%2F' + str(day) + '%2F' + str(year) + '&tbm=nws'
    
    else:
        url = 'https://www.google.com/search?q=' + str(query) + '&sxsrf=ACYBGNTwLQ0U4gEoAyzjmWThzJK06MJhxw%3A1577815826486&source=lnt&tbs=cdr%3A1%2Ccd_min%3A' + str(month) + '%2F' + str(day) + '%2F' + str(year) + '%2Ccd_max%3A' + str(month) + '%2F' + str(day) + '%2F' + str(year) + '&tbm=nws'
        
    return url

# curl = createUrl(query, month=12, day=31, year=2019)
# curl

In [4]:
# defininig a function for getting a range of dates
def dateRange(start, end):
    temp = (end - start).days
    # print(temp)
    for i in range(int(temp)):
        yield (start + timedelta(i))

print(dateRange(start, end))

<generator object dateRange at 0x7ff2a0d47fc0>


In [5]:
# function for getting the headlines using urllib and Beautiful Soup
def captureHeadlines(page):
    
    # A user agent is string that each web browser sends to the website. It contains info.
    # like the device software, OS, etc. so that the remote web servers use this data
    # to optimize the webpage's performance and display.
    userAgent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3579.0 Safari/537.36'
    
    headers = {"User-Agent": userAgent}
    response = requests.get(page, headers = headers)
    # print(response.content)
    
    pubList = ['The Guardian', 'The Independent', 'The New York Times', 'Vox' 
               'Bloomberg', 'The Atlantic', 'USA TODAY', 'Sky Sports', 'Verge', 
               'New Yorker', 'Forbes', 'Nature', 'CNN', 'CNBC', 'BBC News', 'NBC', 
               'Times of India', 'Hindu', 'Economic Times', 'Washington Post',
               'Huffington Post', 'ESPN', 'The Indian Express',
               'Hindustan Times', 'Foreign Policy']
    
    soup = BeautifulSoup(response.content, 'html.parser')
    headlinesCollec = []
    # hdls = soup.find_all('div', attrs={'class': 'bkWMgd'})
    # hdls1 = soup.find_all('div', attrs={'class': 'dbsr'})
    hdls1 = soup.find_all('div', attrs={'class': 'gG0TJc'})
    for data in hdls1:
        print('\n')
        temp = data.find('a')
        pub = data.find('span', attrs={'class': 'xQ82C'}).get_text()

        # dataObj = {
        #     "Title": data.find('div', attrs={'class': 'phYMDf nDgy9d'}).get_text(),
        #     "Publisher": data.find('div', attrs={'class': 'pDavDe RGRr8e'}).get_text(),
        #     "Link": temp['href']
        # }
        dataObj = {
            "Title": data.find('h3', attrs={'class': 'r dO0Ag'}).get_text(),
            "Publisher": pub,
            "Link": temp['href']
        }        
        # another condition to get news only from major news sources
        if (any(pub in j for j in pubList)):
            headlinesCollec.append(dataObj)            
    return headlinesCollec  
    
    # hdls1 = soup.find_all('div', attrs={'class': 'gG0TJc'})
    # for data in hdls1:
    #     print('\n')
    #     temp = data.find('a')
    #     # print(temp)
    #     pub = data.select('.slp .xQ82C')
    #     # pubs = pub.select('.xQ82C')
    #     print(len(pub))
    #     title = data.select('r dO0Ag a')
    #     # titles = title.select('.a')
    #     print(len(title))
    #     dataObj = {
    #         "Title": title.get_text(),
    #         "Publisher": pub.get_text(),
    #         "Link": title['href']
    #         }        
    #         # another condition to get news only from major news sources
    #     if (any(pub in j for j in pubList)):
    #         headlinesCollec.append(dataObj)            
    # return headlinesCollec  


# captureHeadlines('https://www.google.com/search?q=arsenal&biw=1366&bih=669&sxsrf=ACYBGNTLcO_bWTajN0YjR_4rxoHl9GVo7g%3A1577820127297&source=lnt&tbs=cdr%3A1%2Ccd_min%3A12%2F29%2F2019%2Ccd_max%3A12%2F30%2F2019&tbm=nws')


In [6]:
def getHeadlines(query, start, end):
    
    urlList = []
    for dates in dateRange(start,end):
        # print(dates)
        currentDate = dates.strftime("%m-%d-%Y").split('-')
        # print(currentDate)
        url = createUrl(query, currentDate[0], currentDate[1], currentDate[2])
        # print(url)
        urlList.append(url)
        
    headlines = []
    for u in urlList:
        headL = captureHeadlines(u)
        headlines.append(headL)
        
    return headlines

In [7]:
getHeadlines(query, start, end)
# # storing data in JSON format
# import json
# with open('articles.json', 'w', encoding='utf-8') as file:
#     line = json.dumps(content) + '\n'
#     file.write(line)

[[{'Title': 'State will protect interests of indigenous Assamese: CM ...',
   'Publisher': 'Hindustan Times',
   'Link': 'https://www.hindustantimes.com/india-news/state-will-protect-interests-of-indigenous-assamese-cm-sonowal/story-AihOP553yuiXincJwEViJP.html'},
  {'Title': 'Opposition orchestrated anti-CAA protests by misleading the ...',
   'Publisher': 'Hindustan Times',
   'Link': 'https://www.hindustantimes.com/india-news/opposition-orchestrated-anti-caa-protests-by-misleading-the-people-says-naqvi/story-eLINeCFDFGZLZrkxMq9OgI.html'}]]

In [8]:
# print(content)
# table = pd.DataFrame.from_dict(content, orient='index')
# table

















